In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense, Input,Conv1D, MaxPooling1D, Flatten, Embedding, ReLU
from keras.models import Sequential, Model
from keras.models import load_model
import keras

import tensorflow as tf

Using TensorFlow backend.


In [2]:
P = np.array([
    [1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
    [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1],
    [1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
    [1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
    [0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0],
    [0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1],
    [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0],
    [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1],
], dtype='int')

In [3]:
def custom_activation(x):
#     return 1 / (1 + keras.backend.exp((-x)))
    return 10*x*tf.exp(-0.1*tf.square(x))

In [4]:
def awgn(x_input, reqSNR):
    sig_power = 1
    
    noise_power = np.sqrt(sig_power / (2*reqSNR))
#     noise_power = 0.4
    noise = noise_power * np.random.normal(size = x_input.shape)
    
    return x_input + noise

In [5]:
# model = load_model('model_cus_exp.h5', custom_objects={'custom_func':custom_activation})
model = load_model('model_p5_24.h5')
# model = load_model('model_cus_relu.h5')

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 24)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 48)           1200        input_1[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 72)           1800        input_1[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 96)           2400        input_1[0][0]                    
__________________________________________________________________________________________________
dense_13 (

In [ ]:
for run in range(10):
    m = np.random.randint(2, size=(10**7, 12))
    G= np.concatenate((P, np.eye(12, dtype="int")), axis=1)
    x_train = np.dot(m, G) % 2
    x_train = np.where(x_train > 0, -1, 1)
    
    m = np.random.randint(2, size=(10**3, 12))
    x_test = np.dot(m, G) % 2
    x_test = np.where(x_test > 0, -1, 1)    
    
    Eb_No = 10**(3/10)
    Es_No = Eb_No * (12/23)
    x_train_noisy = awgn(x_train[:5*10**6], Es_No)
    x_test_noisy = awgn(x_test[:5*10**2], Es_No)

    Eb_No = 10**(8/10)
    Es_No = Eb_No * (12/23)
    x_train_noisy = np.vstack((x_train_noisy, awgn(x_train[5*10**6:], Es_No)))
    x_test_noisy = np.vstack((x_test_noisy, awgn(x_test[5*10**2:], Es_No)))

    masking = np.ones((10**7, 1))
    masking[5*10**6:] = -1*masking[5*10**6:]
    x_train = np.hstack((masking, x_train))
    x_train_noisy = np.hstack((masking, x_train_noisy))

    masking = np.ones((10**3, 1))
    masking[5*10**2:] = -1*masking[5*10**2:]
    x_test = np.hstack((masking, x_test))
    x_test_noisy = np.hstack((masking, x_test_noisy))
        
    
    model.fit(x_train_noisy, x_train, 
              epochs=100,
              batch_size=10000,
              shuffle=True,
              validation_data=(x_test_noisy, x_test), verbose = 0)
    
    if run%1 == 0:
        loss = model.evaluate(x_test_noisy, x_test)
        print(loss)
        model.save('model_p5_24.h5')
        print('save_model')

1000/1000 [==============================] - 0s 70us/step
[0.03874647109967191, 0.129]
save_model
1000/1000 [==============================] - 0s 72us/step
[0.04309599901229376, 0.128]
save_model
1000/1000 [==============================] - 0s 54us/step
[0.039779116813559084, 0.127]
save_model
1000/1000 [==============================] - 0s 74us/step
[0.0395733229763573, 0.062]
save_model
1000/1000 [==============================] - 0s 50us/step
[0.04271683438424952, 0.087]
save_model
1000/1000 [==============================] - 0s 70us/step
[0.04445242938544834, 0.099]
save_model


In [ ]:
loss = model.evaluate(x_test_noisy, x_test)
print(loss)
model.save('model_p5_24.h5')
print('save_model')